In [1]:
pip install scipy


Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install numpy

Note: you may need to restart the kernel to use updated packages.


In [25]:
import sys
import numpy as np
import scipy

print("NumPy version:", np.__version__)
print("SciPy version:", scipy.__version__)
print("Python version:", sys.version)

NumPy version: 1.26.4
SciPy version: 1.13.1
Python version: 3.9.7 (default, Sep 16 2021, 08:50:36) 
[Clang 10.0.0 ]


In [13]:
n,d,k,tau = 50,12,1,100
W,Lambda = np.ones((d, k)),np.eye(k)
np.random.seed(101) #seed = 41,51,...
z = np.random.normal(0, 1, size=(n, k))
x0= np.dot(W,np.transpose(z))
norm_of_raw_data = np.linalg.norm(x0,ord=2)
x0.shape,norm_of_raw_data


((12, 50), 26.000079468217375)

In [14]:
# Seed
np.random.seed(450)
mean = np.zeros(d)  
cov = (1/tau) * np.eye(d)  
E = np.random.multivariate_normal(mean, cov, size= n).T
# contaminate 
x = x0+E
mu_z = np.full((n, k),0.1)
x.shape, mu_z.shape

((12, 50), (50, 1))

In [6]:
pip install cmdstanpy


     |████████████████████████████████| 94 kB 3.0 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [4]:
import cmdstanpy
cmdstanpy.install_cmdstan()


CmdStan install directory: /Users/arghyadatta/.cmdstan
Installing CmdStan version: 2.35.0
Download successful, file: /var/folders/2x/1nbr4z456238x43xpmlql0nm0000gn/T/tmphu6tw14t
Extracting distribution
Unpacked download as cmdstan-2.35.0
Building version cmdstan-2.35.0, may take several minutes, depending on your system.
Installed cmdstan-2.35.0
Test model compilation


True

In [5]:
print(cmdstanpy.cmdstan_path())


/Users/arghyadatta/.cmdstan/cmdstan-2.35.0


In [1]:
import numpy as np
import cmdstanpy

X = x


stan_data = {
    'n': n,
    'd': d,
    'k': k,
    'X': X
}


stan_model_code = """
data {
  int<lower=1> d;            // number of rows in X
  int<lower=1> n;            // number of columns in X
  int<lower=1> k;            // rank of W and Z
  matrix[d, n] X;            // observed data matrix
}

parameters {
  matrix[d, k] W;            // matrix W of size d x k
  matrix[n, k] Z;            // matrix Z of size n x k
}

model {
  // Priors
  for (j in 1:k)
    W[:, j] ~ normal(0, 1);   // Prior on W
  for (i in 1:n)
    Z[i, ] ~ normal(0, 1);    // Prior on Z

  // Fixed tau = 100
  real tau = 100;

  // Likelihood
  to_vector(X) ~ normal(to_vector(W * Z'), sqrt(1 / tau));
}

generated quantities {
  matrix[d, n] X_hat;
  X_hat = W * Z';             // Posterior predictive mean for X
}
"""

with open('model.stan', 'w') as f:
    f.write(stan_model_code)


model = cmdstanpy.CmdStanModel(stan_file='model.stan')

fit = model.sample(data=stan_data, iter_sampling=2000000, iter_warmup=1000000, chains=4)



posterior = fit.stan_variable('X_hat')
posterior_mean = np.mean(posterior, axis=0)  # Mean of WZ' with respect to the posterior


print("Posterior mean of WZ':")
print(posterior_mean)


NameError: name 'x' is not defined

In [10]:
posterior_mean[:,25]

array([0.37129185, 0.3719278 , 0.38113738, 0.37752904, 0.3752244 ,
       0.37893549, 0.37460343, 0.38092507, 0.37405588, 0.37772616,
       0.37395984, 0.3745464 ])

In [9]:
x0[:,25]

array([0.39052784, 0.39052784, 0.39052784, 0.39052784, 0.39052784,
       0.39052784, 0.39052784, 0.39052784, 0.39052784, 0.39052784,
       0.39052784, 0.39052784])

In [11]:
norm_of_esimated_data = np.linalg.norm((posterior_mean-x0),ord=2)
norm_of_esimated_data

0.6234491052913236

In [8]:
nohup python your_script.py > output.log 2>&1 &


SyntaxError: invalid syntax (1733841062.py, line 1)

In [15]:
import numpy as np
import cmdstanpy

X = x  

stan_data = {
    'n': n,  # Number of columns in X
    'd': d,  # Number of rows in X
    'k': k,  # Rank of W and Z
    'X': X   # Observed data matrix
}

stan_model_code = """
data {
  int<lower=1> d;            // number of rows in X
  int<lower=1> n;            // number of columns in X
  int<lower=1> k;            // rank of W and Z
  matrix[d, n] X;            // observed data matrix
}

parameters {
  matrix[d, k] W;            // matrix W of size d x k
  matrix[n, k] Z;            // matrix Z of size n x k
}

model {
  // Priors
  for (j in 1:k)
    W[:, j] ~ normal(0, 1);   // Prior on W
  for (i in 1:n)
    Z[i, ] ~ normal(0, 1);    // Prior on Z

  // Fixed tau = 100
  real tau = 100;

  // Likelihood
  to_vector(X) ~ normal(to_vector(W * Z'), sqrt(1 / tau));
}

generated quantities {
  matrix[d, n] X_hat;
  X_hat = W * Z';             // Posterior predictive mean for X
}
"""


with open('model.stan', 'w') as f:
    f.write(stan_model_code)


model = cmdstanpy.CmdStanModel(stan_file='model.stan')


fit = model.variational(data=stan_data, algorithm='meanfield', iter=20000)

posterior = fit.stan_variable('X_hat')

print("Shape of posterior:", posterior.shape)

try:
    posterior_mean = posterior.reshape(d, n) 
    print("Posterior mean of WZ' using ADVI:")
    print(posterior_mean)
except ValueError as e:
    print(f"Error reshaping posterior: {e}")
    print("Posterior:", posterior)  


14:39:46 - cmdstanpy - INFO - compiling stan file /Users/arghyadatta/Downloads/Probabilistic-and-Bayesian-PCA-master/model.stan to exe file /Users/arghyadatta/Downloads/Probabilistic-and-Bayesian-PCA-master/model
14:40:01 - cmdstanpy - INFO - compiled model executable: /Users/arghyadatta/Downloads/Probabilistic-and-Bayesian-PCA-master/model
14:40:01 - cmdstanpy - INFO - Chain [1] start processing
14:40:02 - cmdstanpy - INFO - Chain [1] done processing
14:40:02 - cmdstanpy - WARNING - The default behavior of CmdStanVB.stan_variable() will change in a future release to return the variational sample, rather than the mean.
To maintain the current behavior, pass the argument mean=True


Shape of posterior: (12, 50)
Posterior mean of WZ' using ADVI:
[[ 2.68378     0.600433    0.920128    0.468091    0.618438   -0.332976
  -0.880105    0.612425   -1.99322     0.777021    0.525234   -0.581988
   0.204298   -0.711325   -0.946067    0.976658    0.213843    1.93517
   2.57431     0.693747    0.315826    1.67821    -1.67384    -1.12186
  -0.140636    0.378815    0.100681    0.204716    0.83677     0.0530136
   0.601764    0.334502   -0.453476   -0.737807   -0.89534     0.475913
  -0.122034    1.85559     0.228403    1.99072    -1.03271     0.180128
  -1.12585    -0.00713748  0.977121   -0.152066   -0.0453574   0.657325
   2.05782    -0.651598  ]
 [ 2.67418     0.598284    0.916836    0.466416    0.616225   -0.331784
  -0.876956    0.610233   -1.98608     0.774241    0.523355   -0.579906
   0.203567   -0.70878    -0.942682    0.973163    0.213078    1.92824
   2.5651      0.691265    0.314696    1.6722     -1.66785    -1.11784
  -0.140133    0.377459    0.100321    0.203983  

In [16]:
norm_of_error_esimated_data = np.linalg.norm((posterior_mean-x0),ord=2)
norm_of_error_esimated_data

0.675676528654649

In [20]:
posterior_mean.shape

(12, 50)

In [17]:
posterior_mean[:,25]

array([0.378815, 0.377459, 0.394021, 0.384718, 0.385129, 0.397399,
       0.381828, 0.389775, 0.378993, 0.394555, 0.37948 , 0.379217])

In [22]:
import numpy as np
import cmdstanpy

X = x  

stan_data = {
    'n': n,  # Number of columns in X
    'd': d,  # Number of rows in X
    'k': k,  # Rank of W and Z
    'X': X   # Observed data matrix
}

stan_model_code = """
data {
  int<lower=1> d;            // number of rows in X
  int<lower=1> n;            // number of columns in X
  int<lower=1> k;            // rank of W and Z
  matrix[d, n] X;            // observed data matrix
}

parameters {
  matrix[d, k] W;            // matrix W of size d x k
  matrix[n, k] Z;            // matrix Z of size n x k
}

model {
  // Priors
  for (j in 1:k)
    W[:, j] ~ normal(0, 1);   // Prior on W
  for (i in 1:n)
    Z[i, ] ~ normal(0, 1);    // Prior on Z

  // Fixed tau = 100
  real tau = 100;

  // Likelihood
  to_vector(X) ~ normal(to_vector(W * Z'), sqrt(1 / tau));
}

generated quantities {
  matrix[d, n] X_hat;
  X_hat = W * Z';             // Posterior predictive mean for X
}
"""


with open('model.stan', 'w') as f:
    f.write(stan_model_code)


model = cmdstanpy.CmdStanModel(stan_file='model.stan')


fit = model.variational(data=stan_data, algorithm='fullrank', iter=10000)

posterior = fit.stan_variable('X_hat')


print("Shape of posterior:", posterior.shape)


try:
    posterior_mean = posterior.reshape(d, n)  
    print("Posterior mean of WZ' using ADVI:")
    print(posterior_mean)
except ValueError as e:
    print(f"Error reshaping posterior: {e}")
    print("Posterior:", posterior)  # Print the raw posterior for further inspection


16:16:01 - cmdstanpy - INFO - compiling stan file /Users/arghyadatta/Downloads/Probabilistic-and-Bayesian-PCA-master/model.stan to exe file /Users/arghyadatta/Downloads/Probabilistic-and-Bayesian-PCA-master/model
16:16:14 - cmdstanpy - INFO - compiled model executable: /Users/arghyadatta/Downloads/Probabilistic-and-Bayesian-PCA-master/model
16:16:14 - cmdstanpy - INFO - Chain [1] start processing
16:16:15 - cmdstanpy - INFO - Chain [1] done processing
16:16:15 - cmdstanpy - WARNING - The default behavior of CmdStanVB.stan_variable() will change in a future release to return the variational sample, rather than the mean.
To maintain the current behavior, pass the argument mean=True


Shape of posterior: (12, 50)
Posterior mean of WZ' using ADVI:
[[ 2.70542e+00  5.94916e-01  9.16690e-01  4.80278e-01  6.48669e-01
  -3.16406e-01 -8.80171e-01  6.05445e-01 -2.02007e+00  7.83596e-01
   5.26125e-01 -5.66724e-01  2.02049e-01 -7.28705e-01 -9.54017e-01
   9.89200e-01  2.46286e-01  1.95278e+00  2.58507e+00  6.90420e-01
   3.14749e-01  1.69142e+00 -1.69079e+00 -1.12888e+00 -1.41664e-01
   3.69065e-01  1.10503e-01  1.96507e-01  8.31261e-01  8.26300e-02
   6.01870e-01  3.27892e-01 -4.80723e-01 -7.58846e-01 -9.14435e-01
   4.94298e-01 -1.26154e-01  1.90243e+00  2.42482e-01  2.01451e+00
  -1.04492e+00  1.85593e-01 -1.13830e+00  1.06335e-03  9.96131e-01
  -1.59111e-01 -4.96640e-02  6.54341e-01  2.08266e+00 -6.52926e-01]
 [ 2.70134e+00  5.94020e-01  9.15308e-01  4.79554e-01  6.47691e-01
  -3.15929e-01 -8.78844e-01  6.04532e-01 -2.01703e+00  7.82415e-01
   5.25332e-01 -5.65870e-01  2.01745e-01 -7.27607e-01 -9.52579e-01
   9.87709e-01  2.45915e-01  1.94984e+00  2.58117e+00  6.89379e-0

In [23]:
norm_of_error_esimated_data = np.linalg.norm((posterior_mean-x0),ord=2)
norm_of_error_esimated_data

0.6356805576521761

In [24]:
posterior_mean[:,25]

array([0.369065, 0.368509, 0.376813, 0.37397 , 0.371278, 0.373279,
       0.369312, 0.37957 , 0.370147, 0.373109, 0.374975, 0.3698  ])

In [8]:
import numpy as np
import cmdstanpy


X = x


stan_data = {
    'n': n,
    'd': d,
    'k': k,
    'X': X
}



init_W = np.full((d, 1), 0.1)
init_Z = np.full((n, 1), 0.1)


init_values = [
    {'W': init_W, 'Z': init_Z}
]

stan_model_code = """
data {
  int<lower=1> d;            // number of rows in X
  int<lower=1> n;            // number of columns in X
  int<lower=1> k;            // rank of W and Z
  matrix[d, n] X;            // observed data matrix
}

parameters {
  matrix[d, k] W;            // matrix W of size d x k
  matrix[n, k] Z;            // matrix Z of size n x k
}

model {
  // Priors
  for (j in 1:k)
    W[:, j] ~ normal(0, 1);   // Prior on W
  for (i in 1:n)
    Z[i, ] ~ normal(0, 1);    // Prior on Z

  // Fixed tau = 100
  real tau = 100;

  // Likelihood
  to_vector(X) ~ normal(to_vector(W * Z'), sqrt(1 / tau));
}

generated quantities {
  matrix[d, n] X_hat;
  X_hat = W * Z';             // Posterior predictive mean for X
}
"""

with open('model.stan', 'w') as f:
    f.write(stan_model_code)


model = cmdstanpy.CmdStanModel(stan_file='model.stan')

fit = model.sample(
    data=stan_data, 
    iter_sampling=2000000, 
    iter_warmup=1000000, 
    chains=1,
    inits=init_values
)

posterior = fit.stan_variable('X_hat')
posterior_mean = np.mean(posterior, axis=0)  # Mean of WZ' with respect to the posterior

print("Posterior mean of WZ':")
print(posterior_mean)

19:22:47 - cmdstanpy - INFO - compiling stan file /Users/arghyadatta/Downloads/Probabilistic-and-Bayesian-PCA-master/model.stan to exe file /Users/arghyadatta/Downloads/Probabilistic-and-Bayesian-PCA-master/model
19:23:01 - cmdstanpy - INFO - compiled model executable: /Users/arghyadatta/Downloads/Probabilistic-and-Bayesian-PCA-master/model
19:23:01 - cmdstanpy - INFO - CmdStan start processing


chain 1 |          | 00:00 Status

20:38:51 - cmdstanpy - INFO - CmdStan done processing.



Posterior mean of WZ':
[[ 2.68605844  0.59918498  0.91531478  0.46712229  0.64804212 -0.31738265
  -0.88133914  0.59632596 -2.00755578  0.77794907  0.52044017 -0.56936584
   0.20393552 -0.72562583 -0.94455257  0.99322394  0.23809058  1.94559281
   2.57241799  0.69185226  0.32157744  1.68611348 -1.68218334 -1.12893282
  -0.14130245  0.37132556  0.10759578  0.19837986  0.8305506   0.07582987
   0.60943092  0.32533336 -0.46551309 -0.75684976 -0.91203844  0.49802818
  -0.12748307  1.8929572   0.24367337  2.00534001 -1.03908057  0.18806187
  -1.12479088  0.00478931  0.98654605 -0.15540406 -0.0442564   0.66078053
   2.08108262 -0.64629865]
 [ 2.69069138  0.60021572  0.91689738  0.46792705  0.64916212 -0.31793174
  -0.88286099  0.5973525  -2.01101741  0.77929073  0.52133563 -0.57034987
   0.20428535 -0.7268792  -0.94618339  0.9949364   0.23849987  1.9489458
   2.57685472  0.69304436  0.32213252  1.68902323 -1.68508396 -1.13087758
  -0.14154495  0.37196594  0.10778534  0.19872286  0.83198225 

In [9]:
norm_of_error_esimated_data = np.linalg.norm((posterior_mean-x0),ord=2)
norm_of_error_esimated_data

0.6234577329355171

In [10]:
posterior_mean[:,25] #after 3 million iterations

array([0.37132556, 0.37196594, 0.38116951, 0.37756653, 0.37526862,
       0.37897419, 0.37463838, 0.38095018, 0.37409303, 0.3777509 ,
       0.37399093, 0.37457705])

In [ ]:
array([0.37129185, 0.3719278 , 0.38113738, 0.37752904, 0.3752244 ,
       0.37893549, 0.37460343, 0.38092507, 0.37405588, 0.37772616,
       0.37395984, 0.3745464 ])

#after 300,000 iteration

In [12]:
x0[:,25]

array([0.39052784, 0.39052784, 0.39052784, 0.39052784, 0.39052784,
       0.39052784, 0.39052784, 0.39052784, 0.39052784, 0.39052784,
       0.39052784, 0.39052784])